## scVI data integration for healthy PBMC pilot study (Cai 2020 and Cai 2022)

**Objective**: Run scVI data integration model for healthy PBMCs [Cai 2020 and Cai 2022]


- **Developed by**: Mairi McClean

- **Institute of Computational Biology - Computational Health Centre - Helmholtz Munich**

- v230321

- Following this tutorial:https://docs.scvi-tools.org/en/stable/tutorials/notebooks/harmonization.html



In [ ]:
!pip install --quiet scvi-colab
!pip install --quiet scib-metrics
from scvi_colab import install

install()

In [ ]:
import scanpy as sc
import scvi
from rich import print
from scib_metrics.benchmark import Benchmarker
from scvi.model.utils import mde
from scvi_colab import install

In [ ]:
sc.set_figure_params(figsize=(4, 4))

%config InlineBackend.print_figure_kwargs={'facecolor' : "w"}
%config InlineBackend.figure_format='retina'

## Read in datasets for integration
> Cai 2020 + Cai 2022

### Read in datasets

- Read in _Cai Y et al 2020_

In [ ]:
caiy2020 = sc.read_h5ad('/Volumes/Lacie/data_lake/Mairi_example/processed_files/abridged_qc/human/Cai2020_scRNA_PBMC_mm230315_qcd.h5ad')
caiy2020

- Read in _Cai Y et al 2022_

In [ ]:
caiy2022 = sc.read_h5ad('/Volumes/Lacie/data_lake/Mairi_example/processed_files/abridged_qc/human/Cai2022_scRNA_PBMC_mm230315_qcd.h5ad')
caiy2022.obs['status'] = 'active_TB'
caiy2022

### Merge datasets

In [ ]:
caiy_tb = caiy2020.concatenate(caiy2022, batch_key = 'dataset', batch_categories = ['caiy2020', 'caiy2022'], join = 'inner')
caiy_tb

### Check that anndata object only contains PBMC scRNA from healthy donors

In [ ]:
caiy_tb.obs

In [ ]:
caiy_tb.obs['data_type'].value_counts()

In [ ]:
caiy_tb.obs['tissue'].value_counts()

In [ ]:
caiy_tb.obs['status'].value_counts()

In [ ]:
caiy_healthy = caiy_tb[~caiy_tb.obs['status'].isin(['active_TB', 'latent_TB']),:]

In [ ]:
caiy_healthy.obs['status'].value_counts()

### Calculate HVGs

In [ ]:
adata = caiy_healthy.copy()
adata.layers['counts'] = adata.X.copy()

In [ ]:
sc.pp.highly_variable_genes(
    adata,
    flavor = "seurat_v3",
    n_top_genes = 8000,
    layer = "counts",
    batch_key = "sample",
    subset = True
)

### Integration with scVI


In [ ]:
scvi.model.SCVI.setup_anndata(adata, layer="counts", batch_key="batch")

In [ ]:
vae = scvi.model.SCVI(adata, n_layers=2, n_latent=30, gene_likelihood="nb")

In [ ]:
vae.train()

In [ ]:
adata.obsm["X_scVI"] = vae.get_latent_representation()

In [ ]:
sc.pp.neighbors(adata, use_rep="X_scVI")
sc.tl.leiden(adata)

In [ ]:
adata.obsm["X_mde"] = mde(adata.obsm["X_scVI"])

In [ ]:
sc.pl.embedding(
    adata,
    basis="X_mde",
    color=["batch", "leiden"],
    frameon=False,
    ncols=1,
)
sc.pl.embedding(adata, basis="X_mde", color=["cell_type"], frameon=False, ncols=1)

In [ ]:
adata

In [ ]:
caiy_healthy


In [ ]:
adata_export = adata.AnnData(X = caiy_healthy.X, var = caiy_healthy.var, obs = adata.obs, uns = adata.uns, obsm = adata.obsm, layers = caiy_healthy.layers, obsp = adata.obsp)
adata_export

In [ ]:
adata_export.write('/Volumes/LaCie/data_lake/Mairi_example/processed_files/scvi/CaiY_scRNA_healthy_PBMC_mm230321_scVI-clustered.raw.h5ad')